In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import matplotlib.pyplot as plt
# import geopandas as gpd
import numpy as np
import json
from time import time
from geopy.distance import geodesic
from joblib import Parallel, delayed
import pickle as pkl
import seaborn as sns
from datetime import datetime, timedelta
%matplotlib widget
from scipy.spatial import Delaunay
import os
import re


In [2]:
with open('/Users/1moi6/GitHub/Python/Pesquisa/ferrugem-asiatica/modeloml/dados/estacoes.json', 'r') as arquivo:
        estacoes = json.load(arquivo)
colunas = ['data','hora','precipitação','pressão','pressão máxima','pressão mínima','radiação global','temperatura BS','temperatura PO','temperatura máxima','temperatura mínima','temperatura PO máxima','temperatura PO mínima','umidade relativa máxima','umidade relativa mínima','umidade relativa ','direção do vento','vento rajada máxima','velocidade do vento','ocorrências']

In [3]:
dados = pd.read_csv("latlong_pietra.csv")
surtos = pd.read_csv("surtos.csv")

In [6]:
def encontra_baricentro(triangulation, ponto,codigo_estacoes):
    simplex_idx = triangulation.find_simplex(ponto)
    vertices = triangulation.simplices[simplex_idx]
    # if simplex_idx==-1:
    #     return {"z":-999,"coords":[-1,-1,-1],"vertices":vertices}
    
    A = triangulation.points[vertices[0]]
    B = triangulation.points[vertices[1]]
    C = triangulation.points[vertices[2]]
    P = ponto

    # Calcular as coordenadas baricêntricas
    u = ((B[1] - C[1])*(P[0] - C[0]) + (C[0] - B[0])*(P[1] - C[1])) / \
        ((B[1] - C[1])*(A[0] - C[0]) + (C[0] - B[0])*(A[1] - C[1]))

    v = ((C[1] - A[1])*(P[0] - C[0]) + (A[0] - C[0])*(P[1] - C[1])) / \
        ((B[1] - C[1])*(A[0] - C[0]) + (C[0] - B[0])*(A[1] - C[1]))

    w = 1 - u - v
    if np.max(np.abs([u,v,w]))>1:
        return {"coords":[-1,-1,-1],"vertices":[codigo_estacoes[ver] for ver in vertices]}

    return {"coords":[u,v,w],"vertices":[codigo_estacoes[ver] for ver in vertices]}


def interpola(mun):
    for ano in range(2018,2024):
        arquivos = os.listdir(f'/Users/1moi6/GitHub/Python/Pesquisa/ferrugem-asiatica/clima/inmet/{ano}')
        dicio = {a.split("_")[3]:a for a in arquivos}

        pontos = []
        codigo_estacoes = []
        for k in dicio:
            v = estacoes[k]
            x,y = v[4],v[5]
            if x>-70 and y<-30:
                pontos.append([x,y])
                codigo_estacoes.append(k)
        triangulation = Delaunay(pontos)
        res = encontra_baricentro(triangulation, mun,codigo_estacoes)
        pesos = res["coords"]
        vertices = res["vertices"]
        media_mun = pd.DataFrame()
        

        df1 = pd.read_csv(f"/Users/1moi6/GitHub/Python/Pesquisa/ferrugem-asiatica/clima/inmet/{ano}/{dicio[vertices[0]]}",delimiter=';',encoding='latin1',skiprows=8, decimal=",")
        df2 = pd.read_csv(f"/Users/1moi6/GitHub/Python/Pesquisa/ferrugem-asiatica/clima/inmet/{ano}/{dicio[vertices[1]]}",delimiter=';',encoding='latin1',skiprows=8, decimal=",")
        df3 = pd.read_csv(f"/Users/1moi6/GitHub/Python/Pesquisa/ferrugem-asiatica/clima/inmet/{ano}/{dicio[vertices[2]]}",delimiter=';',encoding='latin1',skiprows=8, decimal=",")
        df1.replace({'-9999': np.nan,-9999: np.nan},inplace=True)
        df2.replace({'-9999': np.nan,-9999: np.nan},inplace=True)
        df3.replace({'-9999': np.nan,-9999: np.nan},inplace=True)
        gp1=df1.groupby(df1.columns[0]).mean()
        gp2=df2.groupby(df2.columns[0]).mean()
        gp3=df3.groupby(df3.columns[0]).mean()
        media_aux = pesos[0]*gp1+pesos[1]*gp2+pesos[2]*gp3
        media_aux.columns=colunas[2:]
        media_aux.index.name=colunas[0]
        media_mun = pd.concat([media_mun,media_aux],axis=0)
    
    return media_mun
        


In [7]:
ocorrencias = pd.DataFrame()
for i in range(len(dados)):
    mun = dados.iloc[i,[-2,-1]].str.replace(',', '.').astype(float).to_list()

    media = interpola(mun)
    media["mes"]=media.index.map(lambda x:re.split("-|/",x)[1])
    mm = media.groupby("mes").mean()
    mm["ocorrências"]=surtos.iloc[i,1:-2].to_list()
    mm["município"] = surtos.iloc[i,0]
    ocorrencias = pd.concat([ocorrencias,mm],axis=0)
    print(i)
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139


In [8]:
ocorrencias.to_csv("ocorrencias.csv",index=False)